In [1]:
import os
os.chdir('C:/Users/KAMAR/Videos/ML')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
from matplotlib import pyplot
import tensorflow as tf

import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.io as pio

from keras.preprocessing.sequence import TimeseriesGenerator

import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv('temp.csv')

In [3]:
df['date']=pd.to_datetime(df['date'])

In [4]:
close_data = df['temperature'].values
close_data=close_data.reshape((-1,1))
split_percent = 0.85
split = int(split_percent*len(close_data))
close_train = close_data[:split]
close_test = close_data[split:]
data_train = df['date'][:split]
data_test=df['date'][split:]
print(len(close_train))
print(len(close_test))

6272
1107


In [5]:
df.head()

,date,temperature
0,1999-01-01,29.0
1,1999-01-02,29.5
2,1999-01-03,29.0
3,1999-01-04,29.5
4,1999-01-05,30.5


In [6]:
look_back = 15
train_generator = TimeseriesGenerator(close_train,close_train,length=look_back,batch_size=20)
test_generator = TimeseriesGenerator(close_test,close_test,length=look_back,batch_size=1)

In [7]:
#model_type = 2
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout





model = Sequential()
model.add(LSTM(10,activation='relu',input_shape=(look_back,1)))
#model.Dropout(0.5)
model.add(Dense(1))


model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mse'])

num_epochs = 25
model.fit_generator(train_generator,epochs=num_epochs,verbose=1,shuffle=True)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
313/313 [==============================] - 2s 8ms/step - loss: 247.1547 - mse: 247.1546
Epoch 2/25
313/313 [==============================] - 2s 8ms/step - loss: 2.0978 - mse: 2.0978
Epoch 3/25
313/313 [==============================] - 2s 8ms/step - loss: 1.6515 - mse: 1.6515
Epoch 4/25
313/313 [==============================] - 2s 7ms/step - loss: 1.6422 - mse: 1.6422
Epoch 5/25
313/313 [==============================] - 2s 8ms/step - loss: 1.6380 - mse: 1.6380
Epoch 6/25
313/313 [==============================] - 2s 7ms/step - loss: 1.6061 - mse: 1.6061
Epoch 7/25
313/313 [==============================] - 2s 8ms/step - loss: 1.6125 - mse: 1.6125
Epoch 8/25
313/313 [==============================] - 2s 8ms/step - loss: 1.5756 - mse: 1.5756
Epoch 9/25
313/313 [==============================] - 2s 8ms/step - loss: 1.5676 - mse: 1.5676
Epoch 10/25
313/313 [==============================] - 2s 7ms/ste

In [8]:
prediction = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


In [9]:
close_train=close_train.reshape((-1))
close_test=close_test.reshape((-1))
prediction=prediction.reshape((-1))

In [10]:
tracel = go.Scatter(x=data_train,y=close_train,mode='lines',name='Data')
trace2=go.Scatter(x=data_test,y=prediction,mode='lines',name='Prediction')
trace3= go.Scatter(x=data_test,y=close_test,mode='lines',name='Ground Truth')
layout = go.Layout(title='Weather Forecast',xaxis={'title':'date'},yaxis={'title':'temperature'})
fig = go.Figure(data=[tracel,trace2, trace3], layout=layout)

In [11]:
pio.write_html(fig,file='temperature.html',auto_open=True)

In [12]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 730
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [13]:
#forecast_dates=pd.to_datetime['forecast_dates']

In [14]:
trace1 = go.Scatter(
    x = data_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)

trace2=go.Scatter(x=data_test,y=prediction,
                  mode='lines',name='Prediction')

trace3 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Forecast')
layout= go.Layout(title='Predicted Values',xaxis={'title':'Date'},yaxis={'title':'Temperature'})
fig = go.Figure(data=[tracel,trace2,trace3],layout=layout)

In [15]:
pio.write_html(fig,file='predict.html',auto_open=True)

In [17]:
import numpy as np
temp_=np.asarray(forecast)
np.savetxt('forecast_temp.csv',temp_,delimiter=",")

In [18]:
temp1=np.asarray(forecast)
np.savetxt('forecast_dates_temp.csv',temp1,delimiter=",")